In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
import time

#for dirname, _, filenames in os.walk('/kaggle/input'):
#    for filename in filenames:
#        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import skimage.io
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications.resnet50 import preprocess_input
from tensorflow.keras.preprocessing.image import ImageDataGenerator

from tensorflow.keras.applications.resnet50 import preprocess_input, decode_predictions



In [ ]:
import cv2
import openslide

In [ ]:
from tensorflow.keras.models import model_from_json, load_model

In [ ]:
#os.listdir('../input/panda-model-1')

In [ ]:
base_path = "../input"
#from_fold = '../input/prostate-cancer-grade-assessment/train_images'
from_fold = '../input/prostate-cancer-grade-assessment/test_images'


In [ ]:
model_path ='panda2model' #this is a name of the folder, where we upload the model
model_name = 'model_panda.h5'


In [ ]:
new_model = load_model(os.path.join(base_path, model_path, model_name))

In [ ]:
#def prepare_test_data(test_arr):
#    test_data = []
#    images_names = []
#    for test_name in test_arr:        
#        image_id = test_name.split('.')[0]        
#        test_file = os.path.join(from_fold, test_name)
#        biopsy = skimage.io.MultiImage(test_file)
#        out_file = cv2.resize(biopsy[-1], (512, 512))
#        test_data.append(out_file)
#        images_names.append(image_id)
#        #print(time.time())
#        del biopsy
#    return np.array(images_names), np.array(test_data)

In [ ]:
def prepare_test_data(test_arr,
                       from_fold = from_fold,
                       zoom = 2,
                       tile_square_yside = 50, 
                       tile_square_xside = 50, 
                       tile_fill = 0.3,
                       threshold = 241,
                       transform_ratios = [2.0, 3.0]): #, 5.0, 6.0, 7.0],):
    """
    This is based on 'eliminate_white_direct'
    """
    test_data = []
    images_names = []
    
    for test_name in test_arr:        

        image_id = test_name.split('.')[0]
        images_names.append(image_id)
        
        nonwhite_image_tiles = [] #add tiles to the array
        nonwhite_image_tiles1 = []
        temp_precip = [] #tiles number to abandon 
        image_id = test_name.split('.')[0]        
        test_file = os.path.join(from_fold, test_name)
        biopsy = openslide.OpenSlide(test_file)
        yyy = biopsy.level_dimensions[zoom]
        biopsy = np.array(biopsy.read_region(yyy, zoom, yyy).convert('RGB'))
        #biopsy = skimage.io.MultiImage(test_file)[zoom]   
        try:
            grey_biopsy = cv2.cvtColor(biopsy, cv2.COLOR_BGR2GRAY)
            ret, white_mask = cv2.threshold(grey_biopsy, threshold, 255, 0)
            ytiles = grey_biopsy.shape[0]//tile_square_yside
            xtiles = grey_biopsy.shape[1]//tile_square_xside
            tiles_number = xtiles*ytiles         
            for i in range(ytiles):
                for j in range(xtiles):
                    coordinates = [i*tile_square_yside, (i+1)*tile_square_yside,\
                                       j*tile_square_xside, (j+1)*tile_square_xside]
                    mask_area = white_mask[coordinates[0]:coordinates[1], coordinates[2]:coordinates[3]]
                    if (etalon_field_value - np.sum(mask_area))/etalon_field_value > tile_fill:
                            nonwhite_image_tiles.append(biopsy[coordinates[0]:coordinates[1],\
                                                                  coordinates[2]:coordinates[3]])
                    del coordinates, mask_area
            all_tiles = len(nonwhite_image_tiles) #tiles number
            nonwhite_image_tiles = np.array(nonwhite_image_tiles)
                #tiles configuration:
            for temp_ratio in transform_ratios:
                temp_dim = int(sqrt(all_tiles/temp_ratio))
                temp_precip.append(int(all_tiles - (temp_dim**2*temp_ratio)))
                final_ratio = transform_ratios[np.argmin(temp_precip, axis=0)]
                final_dim = int(sqrt(all_tiles/final_ratio))
                #reshape tiles array and abandon several tiles:
            nonwhite_image_tiles1 = nonwhite_image_tiles[:int(final_dim**2*final_ratio),:,:,:]
            a, b, c, d = nonwhite_image_tiles1.shape
            data_new = (nonwhite_image_tiles1.reshape(int(final_dim), int(final_dim*final_ratio),\
                                                        b, c, d).swapaxes(1,2).reshape(b*int(final_dim),\
                                                        c*int(final_dim*final_ratio), d))            
            biopsy = cv2.resize(data_new, (512, 512))
            try:
                del data_new, grey_biopsy, nonwhite_image_tiles, nonwhite_image_tiles1,\
                temp_precip, ret, white_mask
            except:
                pass
        
        except Exception as ee:
            biopsy = cv2.resize(biopsy, (512, 512))
            
        finally:            
            test_data.append(biopsy)
            del biopsy
    
    return np.array(images_names), np.array(test_data)

In [ ]:
def make_final_labels(from_model_preds):
    return from_model_preds.argmax(axis=-1)

In [ ]:
def make_submission_df(id_array, test_labels):
    d2 = {'image_id': id_array, 'isup_grade': test_labels}
    new_sample = pd.DataFrame(data=d2)
    return new_sample

In [ ]:
if os.path.exists(from_fold):
    #list of test files:
    test_files = os.listdir(from_fold)
    
    #prepare input data for the model:
    test_files_id_array, test_file_data_array = prepare_test_data(test_arr = test_files)
    #predict with the model:
    pred_all = new_model.predict(test_file_data_array)
    #make final predictions:
    final_test_labels = make_final_labels(pred_all)
    
    #make submission:
    submission = make_submission_df(test_files_id_array, final_test_labels)
    submission['isup_grade'] = submission['isup_grade'].astype(int)
    submission.to_csv('submission.csv', index=False)

else: #test data are covered
    sub = pd.read_csv("/kaggle/input/prostate-cancer-grade-assessment/sample_submission.csv")
    sub.to_csv('submission.csv', index=False)
    print('no submission data')